In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import fpfs
import fitsio
import impt
import jax.numpy as jnp
cov_fname = os.path.join(impt.fpfs.__data_dir__, "modes_cov_mat_paper3_045.fits")
cov_mat = fitsio.read(cov_fname)

In [2]:
test_fname = os.path.join(
    impt.fpfs.__data_dir__,
    "fpfs-cut32-0000-g1-0000.fits",
)
test_fname = "/lustre/work/xiangchong.li/work/FPFS2/sim/srcfs3_unif3_cosmo085-var7em3_try3/psf60/fpfs-cut32-0001-g1-0000.fits"


# impt.fpfs
cat = impt.fpfs.read_catalog(test_fname)
# FPFS
data = fitsio.read(test_fname)
cov_mat2 = impt.fpfs.utils.fpfscov_to_imptcov(data)

2023/01/14 21:42:37 ---  Remote TPU is not linked into jax; skipping remote TPU.
2023/01/14 21:42:37 ---  Unable to initialize backend 'tpu_driver': Could not initialize backend 'tpu_driver'
2023/01/14 21:42:37 ---  Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
2023/01/14 21:42:37 ---  Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
2023/01/14 21:42:37 ---  Unable to initialize backend 'tpu': module 'jaxlib.xla_extension' has no attribute 'get_tpu_client'
2023/01/14 21:42:37 ---  Unable to initialize backend 'plugin': xla_extension has no attributes named get_plugin_device_client. Compile TensorFlow with //tensorflow/compiler/xla/python:enable_plugin_device set to true (defaults to false) to enable this.
2023/01/14 21:42:37 ---  No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [3]:
use_sig = False
funcnm = "ts2"
params = impt.fpfs.FpfsParams(
    Const = 20,
    lower_m00=1.5,
    sigma_m00=0.2,
    lower_r2=-0.05,
    upper_r2=1000.,
    sigma_r2=0.5,
    sigma_v=0.2,
)
e1_impt = impt.fpfs.FpfsE1(params, func_name=funcnm)
e2_impt = impt.fpfs.FpfsE2(params, func_name=funcnm)
w_det = impt.fpfs.FpfsWeightDetect(params, func_name=funcnm)
w_sel = impt.fpfs.FpfsWeightSelect(params, func_name=funcnm)

e1_s = e1_impt*w_sel*w_det
e2_s = e2_impt*w_sel*w_det

de1s_dg1 = impt.RespG1(e1_s)
de2s_dg2 = impt.RespG2(e2_s)

In [4]:
selnm = np.array(["detect2", "M00", "R2"])
#selnm = np.array([])
ell_fpfs = fpfs.catalog.fpfs_m2e(data, const=params.Const, noirev=False)
fs = fpfs.catalog.summary_stats(data, ell_fpfs, use_sig=use_sig, ratio=1.0)
fs = impt.fpfs.test_utils.initialize_FPFS(fs, selnm, params)

ell_fpfs2 = fpfs.catalog.fpfs_m2e(data, const=params.Const, noirev=True)
fs2 = fpfs.catalog.summary_stats(data, ell_fpfs2, use_sig=use_sig, ratio=1.0)
fs2 = impt.fpfs.test_utils.initialize_FPFS(fs2, selnm, params)

In [5]:
print(fs.sumE1)
%time print(jnp.sum(e1_s.evaluate(cat)))

-8.539088182277064
-8.539088182277064
CPU times: user 656 ms, sys: 51.2 ms, total: 707 ms
Wall time: 723 ms


In [6]:
print(fs.sumE2)
%time print(jnp.sum(e2_s.evaluate(cat)))

-2.0238737370088904
-2.0238737370088913
CPU times: user 634 ms, sys: 10.4 ms, total: 645 ms
Wall time: 614 ms


In [11]:
bnoise = impt.BiasNoise(e1_s, cov_mat2)
%time print(jnp.sum(bnoise.evaluate(cat)))
print(fs2.sumE1 - fs.sumE1 + fs2.corE1 - fs.corE1)

0.019710821283596552
CPU times: user 2.73 s, sys: 59.5 ms, total: 2.79 s
Wall time: 2.73 s
0.07203644469433967


In [13]:
%time a=e1_s.hessian(cat)

CPU times: user 2.85 s, sys: 117 ms, total: 2.97 s
Wall time: 2.88 s


In [8]:
bnoise = impt.BiasNoise(e1_s, cov_mat)
%time print(jnp.sum(bnoise.evaluate(cat)))
print(fs2.sumE1 - fs.sumE1 + fs2.corE1 - fs.corE1)

1.503245617556784
CPU times: user 2.74 s, sys: 26.4 ms, total: 2.76 s
Wall time: 2.7 s
0.07203644469433967


In [164]:
bnoise = impt.BiasNoise(e2_s, cov_mat2)
print(jnp.sum(bnoise.evaluate(cat)))
print(fs2.sumE2 - fs.sumE2 + fs2.corE2 - fs.corE2)

0.21253593602579096
-0.14050303438970047


In [165]:
bnoise = impt.BiasNoise(e2_s, cov_mat)
print(jnp.sum(bnoise.evaluate(cat)))
print(fs2.sumE2 - fs.sumE2 + fs2.corE2 - fs.corE2)

-0.13002561744247934
-0.14050303438970047


In [166]:
print(fs.sumR1 + fs.corR1)
print(jnp.sum(de1s_dg1.evaluate(cat)))

426.5762868148186
426.1335293516627


In [167]:
print(fs.sumR2 + fs.corR2)
print(jnp.sum(de2s_dg2.evaluate(cat)))

432.8743618852087
433.31615734349873


In [168]:
bnoise = impt.BiasNoise(de1s_dg1, cov_mat2)
print(jnp.sum(bnoise.evaluate(cat)))
print(fs2.sumR1 - fs.sumR1 + fs2.corR1 - fs.corR1)

-17.125833926390726
15.103984980694104


In [169]:
bnoise = impt.BiasNoise(de1s_dg1, cov_mat)
print(jnp.sum(bnoise.evaluate(cat)))
print(fs2.sumR1 - fs.sumR1 + fs2.corR1 - fs.corR1 )

-10.758604706388777
15.103984980694104


In [170]:
bnoise = impt.BiasNoise(de2s_dg2, cov_mat2)
print(jnp.sum(bnoise.evaluate(cat)))
print(fs2.sumR2 - fs.sumR2 + fs2.corR2 - fs.corR2)

-22.799225218492257
15.208755368492042


In [171]:
bnoise = impt.BiasNoise(de2s_dg2, cov_mat)
print(jnp.sum(bnoise.evaluate(cat)))
print(fs2.sumR2 - fs.sumR2 + fs2.corR2 - fs.corR2 )

-28.138323500896824
15.208755368492042


In [172]:
cov_mat3 = cov_mat
cov_mat3[8:, 8:] = 0.
bnoise = impt.BiasNoise(de2s_dg2, cov_mat3)
print(jnp.sum(bnoise.evaluate(cat)))
print(fs2.sumR2 - fs.sumR2 + fs2.corR2 - fs.corR2 )

-28.138323500896824
15.208755368492042


In [173]:
cov_mat3[0,0]

0.2527604999377557

In [174]:
cov_mat2[0,0]

Array(0.2527605, dtype=float64)